In [40]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd

class ImportProductsApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Import Products")
        self.master.geometry("600x400")  # Adjusted window size

        # Create and place widgets
        self.label_select_file = tk.Label(master, text="Select file:")
        self.label_select_file.pack(pady=10)

        self.choose_button = tk.Button(master, text="Choose File", command=self.choose_file)
        self.choose_button.pack(pady=10)

        self.label_select_fields = tk.Label(master, text="Select the fields to be displayed:")
        self.label_select_fields.pack(pady=10)

        self.available_fields = ["Product ID", "Subcategory", "Title", "Price", "Popularity"]
        self.selected_fields = tk.Listbox(master, selectmode=tk.MULTIPLE, height=len(self.available_fields))
        for field in self.available_fields:
            self.selected_fields.insert(tk.END, field)
        self.selected_fields.pack(pady=10)

        self.display_button = tk.Button(master, text="Display Data", command=self.display_data)
        self.display_button.pack(pady=10)

    def choose_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv"), ("JSON files", "*.json")])

        if file_path:
            # Read data from the file
            if file_path.endswith('.csv'):
                self.data = pd.read_csv(file_path)
            else:
                with open(file_path, 'r') as file:
                    df = pd.read_json(file)
                    df4 = pd.json_normalize(df['products'], meta=['subcategory', 'title', 'price', 'popularity'], sep='_')
                    df4['Product ID'] = dict(df['products']).keys()
                    df4.to_csv('output.csv', index=False)
                    self.data = pd.read_csv('output.csv')

    def display_data(self):
        if not hasattr(self, 'data'):
            messagebox.showwarning("Warning", "Please select a file first.")
            return

    # Get selected fields
        selected_indices = self.selected_fields.curselection()
        selected_fields = [self.available_fields[i] for i in selected_indices]

    
    # Always include "Popularity" in displayed columns for sorting
        if "Popularity" not in selected_fields:
            display_columns = selected_fields + ["Popularity"]
        else:
            display_columns = selected_fields

    # Check for duplicate column labels
        if len(set(display_columns)) != len(display_columns):
            messagebox.showwarning("Warning", "Please select unique columns.")
            return

    # Display selected fields
        if display_columns:
            display_data = self.data[display_columns]

        # Sort the DataFrame by "Popularity" in descending order
            display_data = display_data.sort_values(by="Popularity", ascending=False)

        # Drop "Popularity" if it was not originally selected
            if "Popularity" not in selected_fields:
                display_data = display_data.drop(columns=["Popularity"])

            print(display_data)
        else:
            messagebox.showwarning("Warning", "Please select at least one field to display.")




if __name__ == "__main__":
    root = tk.Tk()
    app = ImportProductsApp(root)
    root.mainloop()


     Product ID  Popularity
922        6834       51936
746        5530       48876
429        4340       46198
580        4804       45775
707        5266       44273
..          ...         ...
716        5308         240
56           19         240
930        6884         240
947        7060         240
864        6166         239

[1000 rows x 2 columns]
     Product ID Subcategory  Popularity
922        6834      mobile       51936
746        5530      mobile       48876
429        4340      mobile       46198
580        4804      mobile       45775
707        5266      mobile       44273
..          ...         ...         ...
716        5308      mobile         240
56           19      mobile         240
930        6884      tablet         240
947        7060      mobile         240
864        6166      mobile         239

[1000 rows x 3 columns]
     Product ID Subcategory  Popularity
922        6834      mobile       51936
746        5530      mobile       48876
429        434

KeyboardInterrupt: 